This approach uses the idea of creating a numpy array at first, and then filling it with the necessary components

In [1]:
import pandas as pd
import sys
import numpy as np
import re
import scipy.sparse as sparse

Location = r'../RAW DATA FILES FOR ANALYSIS/FullKDF.kdf'
#Location = r'Mobs-only for JMcL/OTFTPchk_run9_plate8_recovery.kdf'
kdf = pd.read_csv(Location,names=['Key','Val/Col','Row'])
EOSIndexing = kdf[kdf['Key'].str.contains('<EOS>')]
SiteIndexing = kdf[kdf['Key'].str.contains('Site_')]

nosSites = len(SiteIndexing)

SiteInts = nosSites+1

Subsite_NumberOf = 4


##  list variable holding all the properties for which there are only one per subsite/site 

Single_Properties = ['LINMOBIL','VALIDMOBVAL','MAXMOB','POSITIONOFMAX','MAXRANGE','AVERMAX',
                     'OTFT_MOBILITY','VALID_IOFF2','ION@','IONOFF','VTH','SPARE@',
                     'SPARE2','VDS','LENGTH','WIDTH']

##  variable assigned to hold list of all the properties for which there are multiple values 
##  per site/subsite

Swept_Properties = ['V_Gate','I_Gate','V_Drain','I_Drain','I_Source']


# # # # # Determine size of the full array (and create an empty array/df of that size) # # # # #

if nosSites<25:
    DROP = 100
else:
    DROP = nosSites*Subsite_NumberOf
SPAN = Subsite_NumberOf*(2*nosSites+1) + len(Single_Properties)

MAIN_ARRAY_BLANK = np.zeros((DROP,SPAN))

DF_MAIN_ARRAY_BLANK = pd.DataFrame(MAIN_ARRAY_BLANK)

# # # # # # # #  function to find a TERM at any SITE(n) and SUBSITE(k)  # # # # # # # # # # # #

def deviceAndProperties(n,k,term):  
    """
    Returns properties described by 'words' a device (n) and subsite (k)
    """
    
    SiteChoice = kdf.iloc[range(SiteIndexing.index[n-1],EOSIndexing.index[n-1])]  # choose site
    SubsiteChoice = SiteChoice[SiteChoice['Key'].str.contains('subsite%d' % k)] # choose subsite
    termChoice = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)] # allows search for term
    numpyarray = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)]
    fixedarray = numpyarray['Val/Col'].values ## this is of type numpy array    
    return fixedarray

## Attempt to functionalise the single-property column derivation

In [2]:
def seek(df, *words):  #1
    """
    Return a sub-DataFrame of those rows whose Name column match all the words.
    EXAMPLE USAGE: Mobs1 = seek(kdf, 'OTFT_MOBILITY','subsite1')['Val/Col'].values
    """
    spam = kdf[np.logical_and.reduce([kdf['Key'].str.contains(word) for word in words])]
    return spam['Val/Col'].values

def propertybuilder(itemAsAString, searchterm):
    column = []
    item1= pd.DataFrame(seek(kdf, searchterm, 'subsite1'))
    item2= pd.DataFrame(seek(kdf, searchterm, 'subsite2'))
    item3= pd.DataFrame(seek(kdf, searchterm, 'subsite3'))
    item4= pd.DataFrame(seek(kdf, searchterm, 'subsite4'))
    column = pd.concat([item1, item2, item3, item4], axis=0, ignore_index = True)
    column.columns = [itemAsAString]
    return column

site1 = SiteIndexing['Key']
site2 = SiteIndexing['Key']
site3 = SiteIndexing['Key']
site4 = SiteIndexing['Key']
allSites = pd.concat([site1, site2, site3, site4], axis=0, ignore_index = True)

allLENGTHs = propertybuilder('Channel Lengths', 'LENGTH')
allWIDTHs = propertybuilder('Channel Widths', 'WIDTH')
allMAXMOBs = propertybuilder('MaxMobs', 'MAXMOB')
allIONOFFs = propertybuilder('IOnOff', 'IONOFF')
allMOBs = propertybuilder('Mobility', 'OTFT_MOBILITY')
allIOFFs = propertybuilder('IOFF', 'VALID_IOFF2')
allIONs = propertybuilder('IONs', 'ION@')
allCAPs = propertybuilder('Caps', 'SPARE2')
allVDSs = propertybuilder('VDS', 'VDS')
allVTHs = propertybuilder('VTHs', 'VTH')

allSingleProps = pd.concat([allSites, allLENGTHs, allWIDTHs, allMAXMOBs, allIONOFFs, allMOBs, 
                        allIOFFs, allIONs, allCAPs, allVDSs, allVTHs], axis=1, ignore_index = False)
#allSingleProps

In [3]:
VGateSubsite1 = deviceAndProperties(0,1,'V_Gate')  ## need to convert all these to pd.DataFrames if to 
VGateSubsite2 = deviceAndProperties(0,2,'V_Gate')  ## use code below to compile into a single column:
VGateSubsite3 = deviceAndProperties(0,3,'V_Gate')  ## pd.DataFrame(deviceAndProperties(xxxx))
VGateSubsite4 = deviceAndProperties(0,4,'V_Gate')
# allVGates = pd.concat([VGateSubsite1, VGateSubsite2, VGateSubsite3, VGateSubsite4], axis=0, 
# ignore_index = True)
#allVGates.columns = ['VGates']

blank = np.zeros((100,36)) ## Generates BLANK ARRAY HERE
blankdf = pd.DataFrame(blank)

df_1a = pd.DataFrame()
df_1b = pd.DataFrame()
df_2a = pd.DataFrame()
df_2b = pd.DataFrame()
df_3a = pd.DataFrame()
df_3b = pd.DataFrame()
df_4a = pd.DataFrame()
df_4b = pd.DataFrame()

    
df_1a['VGates SS1'] = VGateSubsite1    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
for i in range(1,nosSites+1):
    IDrain_Subsite1 = deviceAndProperties(i,1,'I_Drain')
    df_1a[(i-1)]= IDrain_Subsite1
    Site1a = df_1a.combine_first(blankdf)

for i in range(1,nosSites+1):
    IGate_Subsite1 = deviceAndProperties(i,1,'I_Gate')
    df_1b[(i-1)]= IGate_Subsite1
    Site1b = df_1b.combine_first(blankdf)

#------------------------------------------------------------------------------------------------#

df_2a['VGates SS2'] = VGateSubsite2    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
for i in range(1,nosSites+1):
    IDrain_Subsite2 = deviceAndProperties(i,2,'I_Drain')
    df_2a[(i-1)]= IDrain_Subsite2
    Site2a = df_2a.combine_first(blankdf)

for i in range(1,nosSites+1):
    IGate_Subsite2 = deviceAndProperties(i,2,'I_Gate')
    df_2b[(i-1)]= IGate_Subsite2
    Site2b = df_2b.combine_first(blankdf)
    
#------------------------------------------------------------------------------------------------#
    
df_3a['VGates SS3'] = VGateSubsite3    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
for i in range(1,nosSites+1):
    IDrain_Subsite3 = deviceAndProperties(i,3,'I_Drain')
    df_3a[(i-1)]= IDrain_Subsite3
    Site3a = df_3a.combine_first(blankdf)
    
for i in range(1,nosSites+1):
    IGate_Subsite3 = deviceAndProperties(i,3,'I_Gate')
    df_3b[(i-1)]= IGate_Subsite3
    Site3b = df_3b.combine_first(blankdf)
    
#------------------------------------------------------------------------------------------------#

df_4a['VGates SS4'] = VGateSubsite4    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
for i in range(1,nosSites+1):
    IDrain_Subsite4 = deviceAndProperties(i,4,'I_Drain')
    df_4a[(i-1)]= IDrain_Subsite4
    Site4a = df_4a.combine_first(blankdf)

for i in range(1,nosSites+1):
    IGate_Subsite4 = deviceAndProperties(i,4,'I_Gate')
    df_4b[(i-1)]= IGate_Subsite4
    Site4b = df_4b.combine_first(blankdf)

## Will need to use concat method on the pd object such as
result = pd.concat([Site1a, Site1b, Site2a, Site2b, Site3a, Site3b, Site4a, Site4b, allSingleProps], 
                   axis=1, join='inner', ignore_index = True)
result.head()

C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\pandas\core\index.py:2271: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  return_indexers=return_indexers)
C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\pandas\core\index.py:2271: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  return_indexers=return_indexers)


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,19.9996,4.18498e-013,-1.22351e-012,-1.19812e-012,-3.34797e-013,-3.10073e-008,-4.70234e-014,-2.88246e-013,-8.93664e-015,3.25392e-013,...,40,1005,3.06986756773,-74569910.2688,3.05305496958,4.18498e-013,4.62289e-005,1.11e-008,-1.99997,-1.54823189724
1,18.9792,6.68185e-013,4.18513e-014,-5.97179e-013,-5.76021e-013,-3.07785e-008,2.83072e-013,3.12697e-013,3.21159e-013,1.3453e-012,...,40,1005,2.74850837694,287880151.552,2.73747193586,6.68185e-013,4.10698e-005,1.11e-008,-1.99997,-1.85460210283
2,17.9587,6.51257e-013,-4.27885e-014,6.30108e-014,8.29e-013,-3.0279e-008,4.81975e-013,7.95143e-013,1.02367e-012,8.88247e-013,...,40,1005,4.17860748784,155308455.983,4.14928311847,6.51257e-013,6.11983e-005,1.11e-008,-1.99997,-1.79077196624
3,16.9389,6.08937e-013,1.64579e-013,-5.54847e-014,5.45456e-013,-2.98064e-008,6.72415e-013,8.16303e-013,7.40126e-013,9.30567e-013,...,40,1005,3.37300763987,-256986748.889,3.35113969208,6.08937e-013,5.01247e-005,1.11e-008,-1.99997,-1.22883664944
4,15.9186,7.52825e-013,3.46554e-013,-1.82445e-013,5.11601e-013,-2.97676e-008,9.26335e-013,5.20064e-013,1.2522e-012,1.42148e-012,...,40,1005,3.76896837455,1670.29810312,3.49095835638,7.52825e-013,5.06564e-005,1.11e-008,-1.99997,-3.20690059638


In [4]:
result.to_csv('Full_using_python.csv', index=False, header = False)

# -*-*-*-*-*-*- TEST CELLS - FOR OPTIMISING LATER -*-*-*-*-*-*-

This Cell features the older form for deriving the single property stacked columns; this has been superceded by use of property builder function (above) but as this doesn't use a loop this can likely be optimised further (see commented out section).

### Also need to work in use of list/dict for going through all properties automatically. 

In [ ]:
# # # # # # THIS FUNCTION GETS AROUND THE ISSUE OF FINDING MULTIPLE 40 CHANNEL LENGTHS # # # # #
# # # # # # # # # # # # # # # # ...NEED TO PUT INTO A LOOP...  # # # # # # # # # # # # # 

def seek(df, *words):  #1
    """
    Return a sub-DataFrame of those rows whose Name column match all the words.
    EXAMPLE USAGE: Mobs1 = seek(kdf, 'OTFT_MOBILITY','subsite1')['Val/Col'].values
    """
    spam = kdf[np.logical_and.reduce([kdf['Key'].str.contains(word) for word in words])]
    return spam['Val/Col'].values

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #


#def propertybuilder(itemAsAString, searchterm, column):
#    column = []
#    for i in range(1,5):
#        item_i= pd.DataFrame(seek(kdf, searchterm, 'subsite%d' % i))
#    column = pd.concat([item_1, item_2, item_3, item_4], axis=0, ignore_index = True)
#    column.columns = [itemAsAString]
#    return column

## until bother writing into loop or defining a function, just do for each subsite as such:

Mobs_1 = pd.DataFrame(seek(kdf, 'OTFT_MOBILITY','subsite1'))
Mobs_2 = pd.DataFrame(seek(kdf, 'OTFT_MOBILITY','subsite2'))
Mobs_3 = pd.DataFrame(seek(kdf, 'OTFT_MOBILITY','subsite3'))
Mobs_4 = pd.DataFrame(seek(kdf, 'OTFT_MOBILITY','subsite4'))
allMobs = pd.concat([Mobs1, Mobs2, Mobs3, Mobs4], axis=0, ignore_index = True)
allMobs.columns = ['Mobilities']

IONOFF1 = pd.DataFrame(seek(kdf, 'IONOFF','subsite1'))
IONOFF2 = pd.DataFrame(seek(kdf, 'IONOFF','subsite2'))
IONOFF3 = pd.DataFrame(seek(kdf, 'IONOFF','subsite3'))
IONOFF4 = pd.DataFrame(seek(kdf, 'IONOFF','subsite4'))
allIONOFFs = pd.concat([IONOFF1, IONOFF2, IONOFF3, IONOFF4], axis=0, ignore_index = True)
allIONOFFs.columns = ['IONOFF']

VTH1 = pd.DataFrame(seek(kdf, 'VTH','subsite1'))
VTH2 = pd.DataFrame(seek(kdf, 'VTH','subsite2'))
VTH3 = pd.DataFrame(seek(kdf, 'VTH','subsite3'))
VTH4 = pd.DataFrame(seek(kdf, 'VTH','subsite4'))
allVTHs = pd.concat([VTH1, VTH2, VTH3, VTH4], axis=0, ignore_index = True)
allVTHs.columns = ['VTH']

LENGTH1 = pd.DataFrame(seek(kdf, 'LENGTH','subsite1'))
LENGTH2 = pd.DataFrame(seek(kdf, 'LENGTH','subsite2'))
LENGTH3 = pd.DataFrame(seek(kdf, 'LENGTH','subsite3'))
LENGTH4 = pd.DataFrame(seek(kdf, 'LENGTH','subsite4'))
allLENGTHs = pd.concat([LENGTH1, LENGTH2, LENGTH3, LENGTH4], axis=0, ignore_index = True)
allLENGTHs.columns = ['LENGTHS']


### This cell shows the first work done to attempt generation of a blank array (numpy) which is then filled with the data: this is now used above in the program, but again it needs optimising

In [ ]:
## DONT TOUCH THIS WORKS



jeff = np.zeros((50,50)) # Generate a blank array, of any size - ie for when increasing size of the set
jeffdf = pd.DataFrame(jeff) # assigned to a dataframe

df = pd.DataFrame()

df['VGates SS1'] = VGateSubsite1

for i in range(1,nosSites+1):  # TRY INSERTING BLANK ARRAY LOGIC HERE...
    IDrain_Subsite1 = deviceAndProperties(i,1,'I_Drain')['Val/Col'].values
    #df['I_Drain -- Site',(i,1)] = IDrain_Subsite1
    df[(i-1)]= IDrain_Subsite1
    
    ## Need to figure a way to maintain the column headings for this - need to generate
    ## a blank array where the columns are still labelled!!!

#for i in range(1,nosSites+1):
#    IGate_Subsite1 = deviceAndProperties(i,1,'I_Gate')['Val/Col'].values
#    df['I_Gate -- Site',(i,1)] = IGate_Subsite1

#for n in range(SiteInts):
#df['blank'] = np.nan


#df['VGates SS2'] = VGateSubsite2
#for i in range(1,nosSites+1):
#    IDrain_Subsite2 = deviceAndProperties(i,2,'I_Drain')['Val/Col'].values
#    df['I_Drain -- Site',(i,2)] = IDrain_Subsite2
#df['VGates SS2'] = VGateSubsite2
#for i in range(1,nosSites+1):
#    IGate_Subsite2 = deviceAndProperties(i,2,'I_Drain')['Val/Col'].values
#    df['I_Gate -- Site',(i,2)] = IGate_Subsite2



blah = df.combine_first(jeffdf)

blah

## Will need to use concat method on the pd object such as
## result = pd.concat([df, df1] axis=1, join='inner') - taken from pandas.pydata Merge Join concat...


In [ ]:
df = pd.DataFrame()

df['VGates SS1'] = VGateSubsite1
for i in range(1,nosSites+1):
    IDrain_Subsite1 = deviceAndProperties(i,1,'I_Drain')['Val/Col'].values
    IGate_Subsite1 = deviceAndProperties(i,1,'I_Gate')['Val/Col'].values
    df['I_Drain -- Site',(i,1)] = IDrain_Subsite1
    df['I_Gate -- Site',(i,1)] = IGate_Subsite1
    
df['VGates SS2'] = VGateSubsite2
for i in range(1,nosSites+1):
    IDrain_Subsite2 = deviceAndProperties(i,2,'I_Drain')['Val/Col'].values
    IGate_Subsite2 = deviceAndProperties(i,2,'I_Gate')['Val/Col'].values
    df['I_Drain -- Site',(i,2)] = IDrain_Subsite2
    df['I_Gate -- Site',(i,2)] = IGate_Subsite2
    
df['VGates SS3'] = VGateSubsite3
for i in range(1,nosSites+1):
    IDrain_Subsite3 = deviceAndProperties(i,3,'I_Drain')['Val/Col'].values
    IGate_Subsite3 = deviceAndProperties(i,3,'I_Gate')['Val/Col'].values
    df['I_Drain -- Site',(i,3)] = IDrain_Subsite3
    df['I_Gate -- Site',(i,3)] = IGate_Subsite3
    
df['VGates SS4'] = VGateSubsite4
for i in range(1,nosSites+1):
    IDrain_Subsite4 = deviceAndProperties(i,4,'I_Drain')['Val/Col'].values
    IGate_Subsite4 = deviceAndProperties(i,4,'I_Gate')['Val/Col'].values
    df['I_Drain -- Site',(i,4)] = IDrain_Subsite4
    df['I_Gate -- Site',(i,4)] = IGate_Subsite4

df